## Imports

In [2]:
import os

import pandas as pd
import numpy as np
from scipy import signal as sp
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
%matplotlib widget

# import tensorflow as tf

In [3]:
rootdir = os.getcwd()
os.chdir(rootdir)

## Variable Assignment

In [4]:
gyrFile = 'Gyroscope'
accFile = 'Accelerometer'

normal = 'normal'

upstairs = 'upstairs'
downstairs = 'downstairs'

s1 = 'Smartphone1'
s3 = 'Smartphone3'

# Gait position
gaitpos = ['Normal', 'Upstairs', 'Downstairs']

# Naming convention - gyr(oscope)+Ass(embly)+_s(martphone_location)) and Same for acceleraoemeter
# All the data is stored in a tensor of this assembly list. So, gyrAss{gaitpos}_s{martphone_location}[0, 1, 2, .....] gives the data as sorted by name in the file directory.
# gyrAssNormal_s1[0] gives the gyroscope data of the subject180_sample 01 in the normal gait position.
# gyrAssNormal_s1[1] gives the gyroscope data of the subject180_sample 02 in the normal gait position.
# 
# gyrAssNormal_s[2] gives the gyroscope data of the subject181_sample 01 in the normal gait position.
# 
# and so on. 

# defining arrays for gyroscope data
gyrAssNormal_s1 = []
gyrAssUpstairs_s1 = []
gyrAssDownstairs_s1 = []

gyrAssNormal_s3 = []
gyrAssUpstairs_s3 = []
gyrAssDownstairs_s3 = []

#defining arrays for accelerometer data
accAssNormal_s1 = []
accAssUpstairs_s1 = []
accAssDownstairs_s1 = []

accAssNormal_s3 = []
accAssUpstairs_s3 = []
accAssDownstairs_s3 = []

## Data Parsing

### Parsing Gyroscope Normal Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [5]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if normal in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        gyrAssNormal_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if normal in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        gyrAssNormal_s3.append(gyrData)                        
#End of file data import and processing

### Parsing Gyroscope Upstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [6]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if upstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssUpstairs_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if upstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssUpstairs_s3.append(gyrData)                        
#End of file data import and processing

### Parsing Gyroscope Downstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if downstairs in subdir:
            for file in files:
                if gyrFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssDownstairs_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if downstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssDownstairs_s3.append(gyrData)                        
#End of file data import and processing

### Parsing Accelerometer Normal Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [8]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if normal in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        accAssNormal_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if normal in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        accAssNormal_s3.append(accData)                        
#End of file data import and processing

### Parsing Accelerometer Upstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [9]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if upstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssUpstairs_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if upstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssUpstairs_s3.append(accData)                        
#End of file data import and processing

### Parsing Accelerometer Downstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [10]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if downstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssDownstairs_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if downstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssDownstairs_s3.append(accData)                        
#End of file data import and processing

## Data Validation and Preprocessing

### Check if the data is collected correctly

In [19]:
# Loop through the variables and check if all the data is collected

gaitpos = ['Normal', 'Upstairs', 'Downstairs']

for j in gaitpos:
    for i in range(1, 4, 2):
        for k in range(0, 7):
            gyrl = eval('gyrAss'+str(j) +'_s'+ str(i))
            accl = eval('accAss'+str(j) +'_s'+ str(i))       
            
            '''if np.shape(gyrl) == (10,) and np.shape(accl) == (10,):
                print("Aquired all data from the group")
            else:
                print("Gyroscope " + str(j)+" Smartphone " + str(i)+ ": ", format(np.shape(gyrl)))
                print("Accelerometer " + str(j)+" Smartphone " + str(i)+ ": ", format(np.shape(accl)))
'''
            # print(gyrl[k]['Time (s)'][0])

            freqGyr = 1/(gyrl[k]['Time (s)'][1]-gyrl[k]['Time (s)'][0])
            freqAcc = 1/(accl[k]['Time (s)'][1]-accl[k]['Time (s)'][0])

            # print(f'Frequency of gyrAss{str(j)}_s{str(i)} [{k}] is : {freqGyr}')
            # print(f'Frequency of accAss{str(j)}_s{str(i)} [{k}] is : {freqAcc}')
            # print("\n")

    # print("\n")

    

In [37]:
plt.close()
plt.plot(gyrAssNormal_s1[0].iloc[:,0], gyrAssNormal_s1[0].iloc[:,1], label='Smartphone 1')
# plt.plot(np.zeros_like(gyrAssNormal_s1[0]), linestyle = "--", linewidth = 0.2, dashes=(5, 15), color="black")
plt.legend()
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Filter out the data

#### Lowpass filter (Butterworth)

In [36]:
#Filter the data to remove noise

# inputs for the function is the data, the sampling frequency, the cutoff frequency.

def dataFilter(dataInput, fs=200, fc=5, order = 2):
    Filtrd = np.zeros(dataInput.shape)
    w = fc / (fs * 0.5)
    b, a = sp.butter(fc, w, 'low')
    
    #Filter the X, Y and Z axis of the input data
    for i in range(0, 3):
        Filtrd[:,i] = sp.filtfilt(b,a, dataInput.iloc[:,i+1])

    return Filtrd

xyz = (dataFilter(gyrAssNormal_s1[0], fs = (1/(gyrAssNormal_s1[0]['Time (s)'][1] - gyrAssNormal_s1[0]['Time (s)'][0]))))
xyz1 = (dataFilter(accAssNormal_s1[0], fs = (1/(accAssNormal_s1[0]['Time (s)'][1] - accAssNormal_s1[0]['Time (s)'][0]))))

plt.close()
plt.plot(gyrAssNormal_s1[0].iloc[:,1], label='Raw Data', c = 'y')
plt.plot(xyz[:,0], label='Filtered Data', c = 'b')
plt.plot(np.zeros_like(xyz), linestyle = "--", linewidth = 0.2, dashes=(5, 15), color="black")
plt.legend()
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Chebyshev filter

In [24]:
'''
# chebyshev filter for the accelerometer data
def chebyFilter(dataInput, fs=200, fc=5, order = 2):
    Filtrd = np.zeros(dataInput.shape)
    w = fc / (fs * 0.5)
    b, a = sp.cheby2(order, 20, w, 'low')
    
    #Filter the X, Y and Z axis of the input data
    for i in range(0, 3):
        Filtrd[:,i] = sp.filtfilt(b,a, dataInput.iloc[:,i+1])
    
    return Filtrd
'''

"\n# chebyshev filter for the accelerometer data\ndef chebyFilter(dataInput, fs=200, fc=5, order = 2):\n    Filtrd = np.zeros(dataInput.shape)\n    w = fc / (fs * 0.5)\n    b, a = sp.cheby2(order, 20, w, 'low')\n    \n    #Filter the X, Y and Z axis of the input data\n    for i in range(0, 3):\n        Filtrd[:,i] = sp.filtfilt(b,a, dataInput.iloc[:,i+1])\n    \n    return Filtrd\n"

### Cut the data to the same length

In [49]:
samplingFreq = np.round((1/(gyrAssNormal_s1[0]['Time (s)'][1] - gyrAssNormal_s1[0]['Time (s)'][0])))

100.0

In [54]:
acc_abs = np.linalg.norm(xyz1, axis=1)
plt.close()
plt.plot(acc_abs)
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
def cutData(accFilt, gyrFilt, freq):
    acc_abs = np.linalg.norm(accFilt, axis=1)

    peaks, _ = sp.find_peaks(acc_abs, height = 10, distance=freq*0.5)
    diff_peaks = np.diff(peaks)
    gap1 = np.argmax(diff_peaks[:20])
    gap2 = np.argmax(diff_peaks[-10:])
    gap2 = int(np.shape(diff_peaks)[0] - 10 + gap2)

    acc_cut = accFilt[peaks[gap1+1]:peaks[gap2], :]
    gyr_cut = gyrFilt[peaks[gap1+1]:peaks[gap2], :]

    return acc_cut, gyr_cut

abc1, abc = cutData(xyz1, xyz, np.round((1/(gyrAssNormal_s1[0]['Time (s)'][1] - gyrAssNormal_s1[0]['Time (s)'][0]))))

plt.close()
# plt.plot(xyz[:,0], label='Low passed Gyroscope X', c = 'purple')
plt.plot(abc1[:,1], label='Filtered Data', c = 'orange')
plt.plot(np.zeros_like(abc1), linestyle = "--", linewidth = 0.2, dashes=(5, 15), color="black")
plt.legend()
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plotttt

In [26]:
def gyrPlot(t, y, lbl):
    plt.plot(t, y, label=lbl)

    plt.xlabel('Time (s)')
    plt.ylabel('Angular velocity (rad/s)')
    plt.title('Gyroscope')
    plt.legend()
    plt.show()

def accPlot(t, y, lbl):
    plt.plot(t, y, label=lbl)

    plt.xlabel('Time (s)')
    plt.ylabel('Acceleration (m/s^2)')
    plt.title('Accelerometer')
    plt.legend()
    plt.show()

# gyrPlot(gyrAssUpstairs_s1[5]['Time (s)'], gyrAssUpstairs_s1[5]['X (rad/s)'], 'Smartphone 1')
# gyrPlot(gyrAssUpstairs_s3[5]['Time (s)'], gyrAssUpstairs_s3[5]['Gyroscope x (rad/s)'], 'Smartphone 3')